# Day 26 notebook

The objectives of this notebook are to practice

* the bootstrap
* permutation testing
* computing confidence measures of network features

## Modules used for this assignment

In [1]:
# standard library modules
import random           # for seed and other randomizing functions
import collections      # for Counter

# course modules
import bayesian_network # for the BayesianNetwork class

## Sample data
As some toy data to work with in this notebook, we'll use some stats from the 2022 regular season games for the Green Bay Packers.

In [2]:
# Green Bay Packers 2023 regular season games
# variables: opponent, home/away, packers score, opponent score, packers pass yards, packers rush yards
packers_data = [
    (   'Bears', 'away', 38, 20, 237,  92),
    ( 'Falcons', 'away', 24, 25, 140,  84),
    (  'Saints', 'home', 18, 17, 245,  95),
    (   'Lions', 'home', 20, 34, 203,  27),
    ( 'Raiders', 'away', 13, 17, 175, 110),
    ( 'Broncos', 'away', 17, 19, 194, 137),
    ( 'Vikings', 'home', 10, 24, 196,  74),
    (    'Rams', 'home', 20,  3, 207, 184),
    ('Steelers', 'away', 19, 23, 283, 116),
    ('Chargers', 'home', 23, 20, 295, 102),
    (   'Lions', 'away', 29, 22, 268, 109),
    (  'Chiefs', 'home', 27, 19, 253, 129)]

## PROBLEM 1: The bootstrap (1 POINT)
Implement the `bootstrap` function below which constructs a bootstrapped data set from a given data set.  Recall that a bootstrapped data set is generated by sampling, *with replacement*, observations from the original data set.  Your function should simply be a **single function call** to the appropriate function in Python's [`random`](https://docs.python.org/library/random.html) module.

In [3]:
def bootstrap(data):
    """Returns a bootstrap sample from the data set.
    
    Args:
        data: a list of observations (tuples)
    Returns:
        A list of observations with the same number of observations as in data.
    """
    ###
    ### YOUR CODE HERE
    return random.choices(data, k=len(data))
    ###


In [4]:
# tests for bootstrap
random.seed(42)
assert bootstrap(packers_data) == [
    (    'Rams', 'home', 20,  3, 207, 184),
    (   'Bears', 'away', 38, 20, 237,  92),
    (   'Lions', 'home', 20, 34, 203,  27),
    (  'Saints', 'home', 18, 17, 245,  95),
    ('Steelers', 'away', 19, 23, 283, 116),
    ('Steelers', 'away', 19, 23, 283, 116),
    (   'Lions', 'away', 29, 22, 268, 109),
    ( 'Falcons', 'away', 24, 25, 140,  84),
    ( 'Broncos', 'away', 17, 19, 194, 137),
    (   'Bears', 'away', 38, 20, 237,  92),
    (  'Saints', 'home', 18, 17, 245,  95),
    ( 'Vikings', 'home', 10, 24, 196,  74)]

packers_points = [(obs[2],) for obs in packers_data]
random.seed(1)
assert bootstrap(packers_points) == [
    (24,),
    (29,),
    (23,),
    (20,),
    (17,),
    (17,),
    (20,),
    (23,),
    (24,),
    (38,),
    (29,),
    (17,)]

random.seed(2)
assert bootstrap([("single",)]) == [("single",)]
print("SUCCESS: bootstrap passed all tests!")

SUCCESS: bootstrap passed all tests!


## PROBLEM 2: Permuting (randomizing) data (1 POINT)
Implement the `permute` function below which constructs a permuted version of the given data set.  Recall that permuting a data set involves shuffling the values within each variable (column in our tables).  In your implementation, you should shuffle the values of each variable (column) separately using the [`random.sample`](https://docs.python.org/library/random.html#random.sample) function (not the `random.shuffle` function!), with the first variable shuffled first, the second variable shuffled second, and so on.  Recall that `zip(*seq)` is useful for transposing a data matrix (`seq`) that is represented as a list of tuples/lists.

In [5]:
def permute(data):
    """Returns a permuted (randomized) version of the data set.
    
    Args:
        data: a list of observations (tuples)
    Returns:
        A list of observations with the same number of observations as in data.
    """
    by_variable = zip(*data)
    ###
    ### YOUR CODE HERE
    n = len(data)
    shuffled_by_variable = [random.sample(var_values, n) for var_values in by_variable]
    return list(zip(*shuffled_by_variable))
    ###


In [6]:
# tests for permute
random.seed(42)
assert permute(packers_data) == [
    (   'Lions', 'away', 38, 19, 268,  74),
    ( 'Falcons', 'away', 18, 19, 295, 110),
    (   'Bears', 'home', 10, 20, 194, 184),
    ( 'Raiders', 'home', 17, 17, 203, 137),
    (   'Lions', 'home', 13, 20, 140,  95),
    (  'Chiefs', 'away', 24, 22, 237, 102),
    ( 'Vikings', 'away', 23, 34, 253, 129),
    ('Chargers', 'home', 29, 23, 207,  84),
    ('Steelers', 'away', 27,  3, 245, 116),
    (  'Saints', 'home', 20, 25, 196,  27),
    ( 'Broncos', 'away', 19, 24, 283,  92),
    (    'Rams', 'home', 20, 17, 175, 109)]

packers_points_and_pass_yards = [(obs[2], obs[4]) for obs in packers_data]
random.seed(1)
assert permute(packers_points_and_pass_yards) == [
    (18, 237),
    (23, 196),
    (24, 268),
    (13, 253),
    (29, 207),
    (20, 245),
    (10, 194),
    (17, 140),
    (19, 283),
    (38, 175),
    (27, 203),
    (20, 295)]

print("SUCCESS: permute passed all tests!")

SUCCESS: permute passed all tests!


## Updates to the `BayesianNetwork` class

The `BayesianNetwork` class has now been filled out a bit more to include two key methods/functions:

1. `all_possible_networks`: An iterator over all possible Bayesian Networks structures for the given random variables
2. `BayesianNetwork.model_evidence`: Returns the model evidence: $\log P(D)$

With these two functions, we can now define a function that does a brute force search over all possible networks and returns the network that maximizes the model evidence score:

In [7]:
def best_net(data, possible_nets):
    model_evidences = [net.model_evidence(data) for net in possible_nets]
    return possible_nets[model_evidences.index(max(model_evidences))]

For example, we can use this function to attempt to reconstruct the flight/weather/airline model from a data set simulated from that model.  In this example, the best network turns out to be the true network!

In [8]:
flight_weather_network = bayesian_network.make_flight_weather_network()

all_possible_flight_weather_nets = list(bayesian_network.all_possible_networks(flight_weather_network.vertex_labels(),
                                                                               flight_weather_network.possible_values))

# Generate a data set from the model, as well as a permuted version of this data set
random.seed(1)
flight_weather_dataset = [flight_weather_network.sample() for _ in range(300)]
permuted_flight_weather_dataset = permute(flight_weather_dataset)

# Predict the network structure from this data set using the model evidence score
best_net(flight_weather_dataset, all_possible_flight_weather_nets).plot()

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tce157c78c6ef4ec1b552922b78c8a7a6"> airline flight_status weather

## PROBLEM 3: Feature confidence via the bootstrap (1 POINT)
You are to use the `best_net` function, along with your `bootstrap` function from Problem 1 to compute confidence levels in the features of a network learned from a given data set.  The confidence levels of each feature should be the fraction of bootstrap data sets in which the feature is present in the learned network (the one output by `best_net`).  Your function should take as input a `features_func` function, which extracts the features of interest from a single learned network.  A couple of such feature extraction functions are provided below.  I recommend that you use [`collections.Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) in your implementation.

In [9]:
def extract_edges(net):
    """Extracts the directed edges (with variable names) of the network."""
    return {(net.vertex_label(i), net.vertex_label(j)) for i, j in net.edges()}

def extract_undirected_edges(net):
    """Extracts the undirected edges (with variable names) of the network."""
    return {tuple(sorted(edge)) for edge in extract_edges(net)}

def compute_feature_confidences(data, possible_networks, features_func, num_bootstraps):
    """Computes the bootstrap confidence levels of features in networks learned from the data set.
    Args:
        data: a list of observations (tuples)
        possible_networks: a list of all possible BayesianNetworks
        features_func: a function that returns a set of features present in a learned network
        num_bootstraps: the number of bootstrapped data sets to use
    Returns:
        A dictionary mapping network features to confidence levels (fraction of bootstrap datasets 
        in which the feature is present in the learned network)
    """
    ###
    ### YOUR CODE HERE
    feature_counts = collections.Counter()
    for _ in range(num_bootstraps):
        bootstrap_data = bootstrap(data)
        net = best_net(bootstrap_data, possible_networks)
        feature_counts.update(features_func(net))
    return {feature: count / num_bootstraps for feature, count in feature_counts.items()}
    ###


In [10]:
# tests for compute_feature_confidences

# test with just one bootstrapped data set
random.seed(41)
assert compute_feature_confidences(flight_weather_dataset, 
                                   all_possible_flight_weather_nets,
                                   extract_undirected_edges,
                                   1) == {
    ('airline', 'flight_status'): 1.0, 
    ('flight_status', 'weather'): 1.0}

# test with 10 bootstrapped data sets
random.seed(42)
assert compute_feature_confidences(flight_weather_dataset, 
                                   all_possible_flight_weather_nets,
                                   extract_undirected_edges,
                                   10) == {
    ('flight_status', 'weather'): 1.0,
    ('airline', 'flight_status'): 0.6,
    ('airline',       'weather'): 0.5}

# test with 100 bootstrapped data sets
random.seed(42)
assert compute_feature_confidences(flight_weather_dataset, 
                                   all_possible_flight_weather_nets,
                                   extract_undirected_edges,
                                   100) == {
    ('flight_status', 'weather'): 1.0,
    ('airline', 'flight_status'): 0.71,
    ('airline',       'weather'): 0.41}

# test with a permuted data set, which shouldn't have any real associations
random.seed(42)
assert compute_feature_confidences(permuted_flight_weather_dataset, 
                                   all_possible_flight_weather_nets,
                                   extract_undirected_edges,
                                   100) == {
    ('airline', 'flight_status'): 0.14,
    ('flight_status', 'weather'): 0.67,
    ('airline',       'weather'): 0.15}

print("SUCCESS: compute_feature_confidences passed all tests!")

SUCCESS: compute_feature_confidences passed all tests!


## BONUS ACTIVITIES

1. How much data is needed from the flight/weather/airline model in order for the airline->flight_status edge to be confidently learned?
2. Below is a toy four variable model.  Simulate a data set from this model and then see if you can predict the edges of the model confidently.  There are many more four variable models, so this will take some time to run.
3. Implement another feature extraction function, `markov_blanket_relations`, which outputs a set of pairs of variables (i, j), for which i is in the Markov blanket of j.  Try using it with the toy four variable model.

In [ ]:
four_var_network = bayesian_network.BayesianNetwork(["x1", "x2", "x3", "x4"])
four_var_network.set_cpd("x1", [], [0, 1], 
                         {(): [0.75, 0.25]})
four_var_network.set_cpd("x2", ["x1"], [0, 1],
                         {(0,): [0.9, 0.1],
                          (1,): [0.1, 0.9]})
four_var_network.set_cpd("x3", ["x2"], [0, 1],
                         {(0,): [0.8, 0.2],
                          (1,): [0.2, 0.8]})
four_var_network.set_cpd("x4", ["x3"], [0, 1],
                         {(0,): [0.7, 0.3],
                          (1,): [0.3, 0.7]})
four_var_network.plot()

In [11]:
###
### YOUR CODE HERE
four_var_network = bayesian_network.BayesianNetwork(["x1", "x2", "x3", "x4"])
four_var_network.set_cpd("x1", [], [0, 1], 
                         {(): [0.75, 0.25]})
four_var_network.set_cpd("x2", ["x1"], [0, 1],
                         {(0,): [0.9, 0.1],
                          (1,): [0.1, 0.9]})
four_var_network.set_cpd("x3", ["x2"], [0, 1],
                         {(0,): [0.8, 0.2],
                          (1,): [0.2, 0.8]})
four_var_network.set_cpd("x4", ["x3"], [0, 1],
                         {(0,): [0.7, 0.3],
                          (1,): [0.3, 0.7]})
four_var_network.plot()
###


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0739b51276b04b08bdad8851b831c0b3"> x1 x2 x3 x4